In [ ]:
from ipynb.fs.full.interact_with_app import *

In [ ]:
import pandas as pd
from os import listdir
from os.path import isfile, join

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns
#print("Setup Complete")
#https://stackoverflow.com/questions/66239052/plot-a-seaborn-heatmap-over-a-background-picture
import matplotlib.image as mpimg # add
from io import BytesIO


## ImageTool

In [ ]:
import numpy as np
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

class ImageTool(object):
    def __init__(self, filename=None, image=None, size=None):
        self.filename = filename
        self.image = image

        if self.filename is not None:
            self._load(self.filename)
        
        #if image is None:
        #    self.image = ImageTool.load(filename)
        
        if image is None and size is not None:
            self.image = Image.new('RGB', size)
        
        #ratio = h / self.image.size[1]
        #newsize = (int(ratio * self.image.size[0]), int(ratio * self.image.size[1]) )
        #self.image = self.image.resize([newsize[0],newsize[1]],Image.ANTIALIAS)

    def testload(self):
        #load(filename)
        print(self.image.size)
        display(self.image) 
        #return image
        
    def size(self):
        return self.image.size
        
    def _load(self,filename):
        self.image = Image.open(open(filename, 'rb'))
    
    def crop(self,left, top, right, bottom):  
        try:
            image = self.image.crop((left, top, right, bottom))
            self.image = image
        except IOError:
            print( "cannot create thumbnail for '%s'" % filename)
 
    def drawText(self, text=None, x=None,y=None, size=20, color='red'):
        draw = ImageDraw.Draw(self.image)
        fnt = ImageFont.truetype("DejaVuSans.ttf", size)
        draw.text((x,y), text, font=fnt, fill=color)

    def drawPoint(self, x,y,r,rgba):
        draw = ImageDraw.Draw(self.image)
        leftUpPoint = (x-r, y-r)
        rightDownPoint = (x+r, y+r)
        twoPointList = [leftUpPoint, rightDownPoint]
        draw.ellipse(twoPointList, fill=rgba)

    def drawLine(self,ax,ay,bx,by,color='red',width=1):
        draw = ImageDraw.Draw(self.image)
        draw.line((ax,ay,bx,by), fill=color, width=width)

    def drawBox(self,gx0 , gy0, gx1, gy1,color='white',width=1):
        draw = ImageDraw.Draw(self.image)
        draw.rectangle(((gx0, gy0), (gx1, gy1)), outline=color, width=width)
        
    def drawBBox(self,bbox,color='white',width=1):  
        self.drawBox(bbox[0][0],bbox[0][1], bbox[1][0],bbox[1][1],color=color,width=width)

## PoseImageTool

In [ ]:
class PoseImageTool(ImageTool):
    def __init__(self, filename=None, image=None, size=None, pose=None, minScoreKeypoint=-1):
        
        super(PoseImageTool, self).__init__(filename=filename, image=image, size=size)
        
        self.pose = pose
        #self.xys = get_xys(pose, minScoreKeypoint)
        
    def addPointsText(self,pose=None, color='red',keys=None,radius = 1,fontsize=10):
        self.xys = self.get_xys(pose=pose)
        if keys == None:
            keys = self.xys.keys()
            
        for key in keys:
            if key in self.xys:
                x = self.xys[key][0]
                y = self.xys[key][1]
                self.drawPoint( x,y,radius,color)
                #self.drawText(str(key), x,y,fontsize, color=color)
                
    def addSkeleton(self,pose=None, color='red',width=1):
        self.xys = self.get_xys(pose=pose)
        for a, b in EDGES:
            if a not in self.xys or b not in self.xys: continue
            ax, ay = self.xys[a][0],self.xys[a][1]
            bx, by = self.xys[b][0],self.xys[b][1]
            self.drawLine(ax,ay,bx,by,color=color,width=width)
                
    def printPose(self,mninScorePose=-0.1, minScoreKeypoint=-0.1):
        #if self.pose.score < mninScorePose : continue
        print('\nPose Score: ', self.pose.score)
        for label, keypoint in self.pose.keypoints.items():
            if keypoint.score < minScoreKeypoint : continue
            print(' %-20s x=%-4d y=%-4d score=%.2f' %
                (label, keypoint.yx[1], keypoint.yx[0], keypoint.score))
            
    def get_xys(self, pose=None, min_score=-1):
        xys = {}
        for label, keypoint in pose.keypoints.items():
            score =  float(keypoint.score)
            if score < min_score: continue
            kp_x = int((keypoint.yx[0] ) )
            kp_y = int((keypoint.yx[1] ) )
            xys[label] = (kp_x, kp_y, score)
            #print(' %-20s x=%-4d y=%-4d score=%.1f' %
            #          (label, keypoint.yx[1], keypoint.yx[0], keypoint.score))
        return xys               

## PoseBox

In [ ]:
import collections
import math
import os
import time

import numpy as np

from PIL import Image

EDGES = (
    ('nose', 'left eye'),
    ('nose', 'right eye'),
    ('nose', 'left ear'),
    ('nose', 'right ear'),
    ('left ear', 'left eye'),
    ('right ear', 'right eye'),
    ('left eye', 'right eye'),
    ('left shoulder', 'right shoulder'),
    ('left shoulder', 'left elbow'),
    ('left shoulder', 'left hip'),
    ('right shoulder', 'right elbow'),
    ('right shoulder', 'right hip'),
    ('left elbow', 'left wrist'),
    ('right elbow', 'right wrist'),
    ('left hip', 'right hip'),
    ('left hip', 'left knee'),
    ('right hip', 'right knee'),
    ('left knee', 'left ankle'),
    ('right knee', 'right ankle'),
)

KEYPOINTS = (
  'nose',
  'left eye',
  'right eye',
  'left ear',
  'right ear',
  'left shoulder',
  'right shoulder',
  'left elbow',
  'right elbow',
  'left wrist',
  'right wrist',
  'left hip',
  'right hip',
  'left knee',
  'right knee',
  'left ankle',
  'right ankle'
)

BODYPIX_PARTS = {
  0: "left face",
  1: "right face",
  2: "left upper arm front",
  3: "left upper arm back",
  4: "right upper arm front",
  5: "right upper arm back",
  6: "left lower arm front",
  7: "left lower arm back",
  8: "right lower arm front",
  9: "right lower arm back",
  10: "left hand",
  11: "right hand",
  12:  "torso front",
  13:  "torso back",
  14:  "left upper leg front",
  15:  "left upper leg back",
  16:  "right upper leg front",
  17:  "right upper leg back",
  18:  "left lower leg front",
  19:  "left lower leg back",
  20:  "right lower leg front",
  21:  "right lower leg back",
  22:  "left feet",
  23:  "right feet",
}

class Keypoint:
    __slots__ = ['k', 'yx', 'score']

    def __init__(self, k, yx, score=None):
        self.k = k
        self.yx = yx
        self.score = score

    def __repr__(self):
        return 'Keypoint(<{}>, {}, {})'.format(KEYPOINTS[self.k], self.yx, self.score)


class Pose:
    __slots__ = ['keypoints', 'score']

    def __init__(self, keypoints, score=None):
        assert len(keypoints) == len(KEYPOINTS)
        self.keypoints = keypoints
        self.score = score

    def __repr__(self):
        return 'Pose({}, {})'.format(self.keypoints, self.score)

    
class BaseBox(object):   
    def __init__(self,size=None, min=None, max=None ):
        self.size = size
        self.min = min
        self.max = max
        
    def ctime(self):
        return time.ctime(self.time_seconds)
    
    def time(self):
        return time.localtime(self.time_seconds)
    
    def asctime(self):
        return self.ctime().split(' ', 1)[1].replace(' ','-')    
    
class PoseBox(BaseBox):
    def __init__(self, keypoints=None, time_seconds=None, score_range=None, bbox=None, size=None, min=None, max=None ):
        
        super(PoseBox, self).__init__(size=size, min=min, max=max)
        
        #self.score     = keypoints[1]
        self.score_range = score_range
        self.time_seconds = time_seconds
        self.bbox = bbox
        
        self.pose = Pose(self.init_keypoints(keypoints=keypoints[0]), keypoints[1])
        #self.init_keypoints(keypoints=keypoints[0]) 
        
    def init_keypoints(self,keypoints=None):        
        keypoint_dict = {}
        for key,point in keypoints.items():    
            x = point[0][0]
            y = point[0][1]
            score = point[1]
            keypoint = Keypoint(int(key), (x, y), score)
            keypoint_dict[KEYPOINTS[int(key)]] = keypoint
        return keypoint_dict    

## ObjectBox

In [ ]:
class ObjectBox(BaseBox):
    def __init__(self, data=None, inference_time=None, time_seconds=None, size=None, min=None, max=None ):  
        
        super(ObjectBox, self).__init__(size=size, min=min, max=max)
        
        self.id = data[0]
        self.score =  data[1]
        self.bbox = ((data[2][0],data[2][1]),(data[2][2],data[2][3]))
        self.inference_time = inference_time
        self.time_seconds = time_seconds

## Functions

In [ ]:
def box_to_bbox_world(x=None,y=None, size = (1920,1080), grid_size=(12,6)):
    step = (int(size[0] / grid_size[0]),int(size[1] / grid_size[1]))
    min = (x*step[0], y*step[1])
    max = ((x+1)*step[0]-1,(y+1)*step[1]-1)
    return min,max


def get_poses_for_filename(path=None,filename=None):
    _size = (1920,1080) 
    _grid_size=(12,6)

    _poses = []
    Box=filename.data(path=path)
    y=filename.y
    x=filename.x
 
    min,max= box_to_bbox_world(x=x,y=y)
    
    for index in range (0,len(Box[4])):
        _poses.append(PoseBox(keypoints=Box[4][index], time_seconds=Box[0], score_range=Box[3][index], bbox=Box[5][index] , size=_size,min=min, max=max))    
    return _poses

def get_objects_for_filename(path=None,filename=None):
    _size = (1920,1080) 
    _grid_size=(12,6)

    Box=filename.data(path=path)
    y=filename.y
    x=filename.x
 
    min,max= box_to_bbox_world(x=x,y=y)
    _objects = []
    for index in range (0,len(Box[3])):
        _objects.append(ObjectBox(data=Box[3][index], inference_time=float(Box[4]), time_seconds=Box[0] , size=_size,min=min, max=max))    
    return _objects


def add_object_to_image(_image_tool,_object,index):
    _image_tool.drawBBox(_object.bbox)
    
    x=_object.bbox[0][0]
    y=_object.bbox[0][1]
    fontsize=40
    _image_tool.drawText(str(index), x,y,fontsize, color='blue')    
    

        
def add_pose_to_image(_image_tool,_pose,index):
    _image_tool.drawBBox(_pose.bbox)
    
    _image_tool.addSkeleton(pose=_pose.pose)
    _image_tool.addPointsText(pose=_pose.pose,radius=2)
    
    x=_pose.bbox[0][0]
    y=_pose.bbox[1][1]
    fontsize=30
    _image_tool.drawText(str(index), x,y,fontsize, color='blue')         
    

## dataframes and heatmaps

In [ ]:

def change_column_type(df,columns,type):
    for column in columns:
        df[column] = df[column].astype(type)

def create_empty_dataframe():
    df = pd.DataFrame(columns=list(range(0, 12)))
    for i in range(6):
        df.loc[i] = listofzeros = [0] * 12
    return df.apply(pd.to_numeric)        

def empty_objects_dataframe():  
    df = pd.DataFrame(columns = ['key','size','grid_box','x', 'y', 'id', 'score', 'bbox', 'inference_time', 'time_seconds','year','month','day','hour','min','sec' ])
    change_column_type(df,['x','y','year','month','day','hour','min','sec'],int)
    return df

def add_objets_to_dataframe(df,_key,_object_boxes,size=(162, 181),black=(211,0)):
    for box in _object_boxes:
        _time = time.localtime(box.time_seconds)
        _grid_box = bbox_to_grid(bbox=box.bbox,min=box.min,max=box.max,size=size,black=black)
        df.loc[len(df.index)] = [_key,box.size,(box.min,box.max),int(box.x),box.y,box.id,box.score,_grid_box,box.inference_time,box.time_seconds,_time.tm_year,_time.tm_mon,_time.tm_mday,_time.tm_hour, _time.tm_min, _time.tm_sec]

def empty_poses_dataframe():     
    df = pd.DataFrame(columns = ['key','size','grid_box','x', 'y', 'id', 'score', 'bbox', 'inference_time', 'time_seconds','year','month','day','hour','min','sec','pose' ])
    change_column_type(df,['x','y','year','month','day','hour','min','sec'],int)
    return df
    
## FIXME _pose.time_seconds different to _filename.time_seconds
def add_poses_to_dataframe(df,_key,_filename,_poses,size=(162, 181),black=(211,0)):
    for _pose in _poses:
        #_time = time.localtime(_pose.time_seconds)
        _time = time.localtime(_filename.time_seconds)
        _grid_box = bbox_to_grid(bbox=_pose.bbox,min=_pose.min,max=_pose.max,size=size,black=black)
        df.loc[len(df.index)] = [_key,_pose.size,(_pose.min,_pose.max),int(_filename.x),_filename.y,'person',_pose.pose.score,_grid_box,None,_filename.time_seconds,_time.tm_year,_time.tm_mon,_time.tm_mday,_time.tm_hour, _time.tm_min, _time.tm_sec,_pose.pose]
        

In [ ]:
def draw_dataframe_grid_image_object_boxes(size=(162, 181), black=(211,0)):    
    for _box in _object_boxes:
        _image_tool.drawBBox((_box.min,_box.max))
        _image_tool.drawText(str(_box.y) + '_' + str(_box.x) , bbox[0][0] + _box.min[0], bbox[1][1] + _box.min[1] -50,fontsize, color='white')  
        if _box.score > 0.5:
            _image_tool.drawBBox(_grid_box, color='red')
            
def draw_dataframe_grid_image(_image_tool,df):
    for _box in pd.unique(df['grid_box']):    
        _image_tool.drawBBox(_box)    
        _row = df[df['grid_box'] == _box].iloc[0]
        _min=_row['grid_box'][0]
        _max=_row['grid_box'][1]
        fontsize=40
        _image_tool.drawText(str(_row['y']) + '_' + str(_row['x']) , _min[0], _max[1] - 50,fontsize, color='white')

    for i in range(0, len(df.index)):
        _image_tool.drawBBox(df.iloc[i]['bbox'], color='red')            

In [ ]:
# for testing
def create_heatmap_save(dataframe=None,title=None,vmin=0,vmax=16):
    # Set the width and height of the figure
    plt.figure(figsize=(14,7))
    # Add title
    plt.title(title)
    # Heatmap showing average arrival delay for each airline by month
    hm = sns.heatmap(data=dataframe, annot=True, vmin=vmin,vmax=vmax)
    # Add label for horizontal axis
    plt.xlabel("")
    plt.savefig('hm.png')
    return hm

def create_heatmap_overlay(image=None,figsize=(19.2,10.8),dataframe=None,annot=True,alpha=0.4,vmin=0,vmax=16):
    _out=widgets.Output()
    with _out:
        plt.figure(figsize=figsize)

        h = sns.heatmap(data=dataframe, alpha=alpha, zorder=2,annot=annot, vmin=vmin,vmax=vmax) # update
        h.imshow(image,
                 aspect=h.get_aspect(),
                 extent= h.get_xlim() + h.get_ylim(),
                 zorder=1)

        buffer1 = BytesIO()
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)

        plt.show() 

        im = Image.open(buf)
        #buf.close()
        return im